In [15]:
import pandas as pd
import numpy as np

In [16]:
df=pd.read_csv('../data/raw/BreastCancerDS.csv',index_col=0)


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19710 entries, 0 to 19709
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   index                    19710 non-null  int64 
 1   perimeter_se             19015 non-null  object
 2   radius_worst             19085 non-null  object
 3   concave points_mean      18980 non-null  object
 4   smoothness_mean          18825 non-null  object
 5   area_mean                18830 non-null  object
 6   concavity_se             18905 non-null  object
 7   texture_mean             18955 non-null  object
 8   concavity_worst          19055 non-null  object
 9   smoothness_se            19110 non-null  object
 10  concave points_se        19000 non-null  object
 11  area_worst               19060 non-null  object
 12  compactness_mean         18935 non-null  object
 13  radius_mean              19000 non-null  object
 14  area_se                  18895 non-nul

## Limpiando  el target

In [18]:
df['diagnosis'].value_counts()

B                          10245
M                           5935
999765432456788              655
-88888765432345              655
?                            655
rxctf378968 7656463sdfg      655
Name: diagnosis, dtype: int64

In [19]:
df=df[(df['diagnosis']=='B')|(df['diagnosis']=='M')]


## Borrando las columnas iuytr y erty

In [20]:
df['erty'].value_counts() ## todos los valores son iguales, se borra erty
df.drop('erty',axis=1,inplace=True)

In [21]:
df[['iuytr','symmetry_mean']] ## estas dos columnas tienen la misma informacion, borrar una
df.drop('iuytr',axis=1,inplace=True)

In [22]:
df

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,id,symmetry_mean,symmetry_worst,diagnosis,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
1,279,1.83,999765432456788.0,0.03711,0.09516,587.4,0.01457,15.18,0.1456,0.004235,...,8911834,211.0,0.2955,B,0.001593,999765432456788.0,0.1724,0.01528,0.01541,999765432456788.0
2,307,1144.0,9699.0,0.003472,-88888765432345.0,246.3,0.003681,14.4,0.01472,0.007389,...,89346,-88888765432345.0,0.2991,B,0.002153,56.36,0.05232,0.02701,0.004883,0.1746
4,576,2183.0,?,0.02278,0.09524,409.0,0.01349,18.75,?,0.008328,...,85759902,?,0.3306,B,0.002386,73.34,?,0.03218,0.008722,0.3249
5,350,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,899187,0.1671,0.2731,B,0.002668,73.7,0.06476,0.02216,0.006991,0.3534
7,120,1103.0,12.82,0.02623,0.09373,403.3,0.01514,10.82,0.2102,?,...,865137,0.1667,0.3016,B,0.002206,73.34,239.0,0.01344,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19703,602,2844.0,?,0.05613,0.1008,809.8,0.02219,21.54,0.2992,0.004877,...,86730502,216.0,?,M,?,106.2,0.3055,0.01535,0.01952,0.4332
19705,350,2225.0,rxctf378968 7656463sdfg,0.01162,0.07561,421.0,0.005949,17.07,0.03046,rxctf378968 7656463sdfg,...,899187,0.1671,0.2731,B,rxctf378968 7656463sdfg,73.7,0.06476,0.02216,0.006991,0.3534
19706,442,2235.0,15.27,0.009937,-88888765432345.0,585.9,0.007741,15.79,0.03517,-88888765432345.0,...,90944601,0.1405,0.1859,B,0.002564,-88888765432345.0,0.1071,-88888765432345.0,0.01156,0.3563
19708,501,2974.0,16.01,0.06759,0.1162,595.9,0.03476,24.49,0.3381,0.00968,...,91504,0.2275,0.3651,M,0.006995,92.33,0.3966,0.02434,0.03856,0.4751


In [23]:
missing = df.isnull().sum()
missing[missing>0]*100/len(df)

perimeter_se               3.275649
radius_worst               3.182942
concave points_mean        3.244747
smoothness_mean            4.295426
area_mean                  4.140915
concavity_se               3.831891
texture_mean               3.522868
concavity_worst            3.213844
smoothness_se              2.626700
concave points_se          3.430161
area_worst                 3.337454
compactness_mean           3.770087
radius_mean                3.491965
area_se                    3.831891
concave points_worst       2.997528
fractal_dimension_worst    2.904821
perimeter_worst            3.152040
texture_se                 3.182942
fractal_dimension_mean     3.090235
texture_worst              3.244747
smoothness_worst           3.337454
concavity_mean             4.295426
id                         2.657602
symmetry_mean              3.831891
symmetry_worst             2.843016
fractal_dimension_se       3.059333
perimeter_mean             3.770087
compactness_worst          2

In [24]:
df.dropna()


,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,id,symmetry_mean,symmetry_worst,diagnosis,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
1,279,1.83,999765432456788.0,0.03711,0.09516,587.4,0.01457,15.18,0.1456,0.004235,...,8911834,211.0,0.2955,B,0.001593,999765432456788.0,0.1724,0.01528,0.01541,999765432456788.0
2,307,1144.0,9699.0,0.003472,-88888765432345.0,246.3,0.003681,14.4,0.01472,0.007389,...,89346,-88888765432345.0,0.2991,B,0.002153,56.36,0.05232,0.02701,0.004883,0.1746
4,576,2183.0,?,0.02278,0.09524,409.0,0.01349,18.75,?,0.008328,...,85759902,?,0.3306,B,0.002386,73.34,?,0.03218,0.008722,0.3249
5,350,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,899187,0.1671,0.2731,B,0.002668,73.7,0.06476,0.02216,0.006991,0.3534
7,120,1103.0,12.82,0.02623,0.09373,403.3,0.01514,10.82,0.2102,?,...,865137,0.1667,0.3016,B,0.002206,73.34,239.0,0.01344,?,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19703,602,2844.0,?,0.05613,0.1008,809.8,0.02219,21.54,0.2992,0.004877,...,86730502,216.0,?,M,?,106.2,0.3055,0.01535,0.01952,0.4332
19705,350,2225.0,rxctf378968 7656463sdfg,0.01162,0.07561,421.0,0.005949,17.07,0.03046,rxctf378968 7656463sdfg,...,899187,0.1671,0.2731,B,rxctf378968 7656463sdfg,73.7,0.06476,0.02216,0.006991,0.3534
19706,442,2235.0,15.27,0.009937,-88888765432345.0,585.9,0.007741,15.79,0.03517,-88888765432345.0,...,90944601,0.1405,0.1859,B,0.002564,-88888765432345.0,0.1071,-88888765432345.0,0.01156,0.3563
19708,501,2974.0,16.01,0.06759,0.1162,595.9,0.03476,24.49,0.3381,0.00968,...,91504,0.2275,0.3651,M,0.006995,92.33,0.3966,0.02434,0.03856,0.4751


In [25]:
table = pd.pivot_table(df, index =["index","id"])

C:\Users\CAMARA22\AppData\Local\Temp\ipykernel_20676\155383082.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  table = pd.pivot_table(df, index =["index","id"])


In [26]:
table


Empty DataFrame
Columns: []
Index: [(0, 842302), (0, rxctf378968 7656463sdfg), (1, 842517), (1, 842517.0), (2, 84300903), (3, 84348301), (3, ? ), (3, rxctf378968 7656463sdfg), (4, 84358402), (5, 843786), (6, 844359), (6, 844359.0), (6, 999765432456788), (7, -88888765432345), (7, 84458202), (7, 84458202.0), (7, 999765432456788), (8, 844981), (8, 844981.0), (8, ? ), (9, 84501001), (9, 84501001.0), (9, rxctf378968 7656463sdfg), (10, 845636), (11, 84610002), (11, 84610002.0), (12, 846226), (12, 846226.0), (12, 999765432456788), (12, ? ), (13, 846381), (13, 846381.0), (13, ? ), (14, 84667401), (14, 84667401.0), (15, 84799002), (15, rxctf378968 7656463sdfg), (16, 848406), (17, -88888765432345), (17, 84862001), (17, 84862001.0), (17, ? ), (18, 849014), (18, 849014.0), (19, 8510426), (19, 8510426.0), (20, 8510653), (20, 8510653.0), (20, rxctf378968 7656463sdfg), (21, 8510824), (22, 8511133), (22, 8511133.0), (23, 851509), (23, 851509.0), (23, 999765432456788), (23, ? ), (24, 852552), (24, 852552.0), (25, 852631), (25, 852631.0), (26, 852763), (26, 852763.0), (26, ? ), (27, 852781), (27, 852781.0), (28, 852973), (28, 852973.0), (28, 999765432456788), (29, 853201), (29, 999765432456788), (29, ? ), (30, 853401), (30, 853401.0), (30, 999765432456788), (31, 853612), (31, ? ), (32, 85382601), (32, 85382601.0), (32, ? ), (33, 854002), (33, 854002.0), (34, 854039), (34, 854039.0), (35, 854253), (35, 854253.0), (35, 999765432456788), (36, 854268), (36, 854268.0), (36, ? ), (37, 854941), (37, 854941.0), (38, 855133), (38, 855133.0), (38, ? ), (39, 855138), (39, 855138.0), (39, 999765432456788), (40, 855167), (40, 855167.0), (41, -88888765432345), ...]

[1490 rows x 0 columns]

In [27]:
df['index']=df['index'].astype('string')
list= ['index', 'perimeter_se', 'radius_worst', 'concave points_mean',
       'smoothness_mean', 'area_mean', 'concavity_se', 'texture_mean',
       'concavity_worst', 'smoothness_se', 'concave points_se', 'area_worst',
       'compactness_mean', 'radius_mean', 'area_se', 'concave points_worst'
       , 'fractal_dimension_worst', 'perimeter_worst', 'texture_se',
       'fractal_dimension_mean', 'texture_worst', 'smoothness_worst',
       'concavity_mean', 'id', 'symmetry_mean', 'symmetry_worst', 'diagnosis',
       'fractal_dimension_se', 'perimeter_mean', 'compactness_worst',
       'symmetry_se', 'compactness_se', 'radius_se']
for i in list:
    df[i]= df[i].str.strip()

In [28]:
list=['rxctf378968 7656463sdfg','-88888765432345.0','999765432456788.0','?']
for i in list:
       df.replace(i,np.nan,inplace=True)


In [29]:
df

,index,perimeter_se,radius_worst,concave points_mean,smoothness_mean,area_mean,concavity_se,texture_mean,concavity_worst,smoothness_se,...,id,symmetry_mean,symmetry_worst,diagnosis,fractal_dimension_se,perimeter_mean,compactness_worst,symmetry_se,compactness_se,radius_se
1,279,1.83,NaN,0.03711,0.09516,587.4,0.01457,15.18,0.1456,0.004235,...,8911834,211.0,0.2955,B,0.001593,NaN,0.1724,0.01528,0.01541,NaN
2,307,1144.0,9699.0,0.003472,NaN,246.3,0.003681,14.4,0.01472,0.007389,...,89346,NaN,0.2991,B,0.002153,56.36,0.05232,0.02701,0.004883,0.1746
4,576,2183.0,NaN,0.02278,0.09524,409.0,0.01349,18.75,NaN,0.008328,...,85759902,NaN,0.3306,B,0.002386,73.34,NaN,0.03218,0.008722,0.3249
5,350,2225.0,13.28,0.01162,0.07561,421.0,0.005949,17.07,0.03046,0.006583,...,899187,0.1671,0.2731,B,0.002668,73.7,0.06476,0.02216,0.006991,0.3534
7,120,1103.0,12.82,0.02623,0.09373,403.3,0.01514,10.82,0.2102,NaN,...,865137,0.1667,0.3016,B,0.002206,73.34,239.0,0.01344,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19703,602,2844.0,NaN,0.05613,0.1008,809.8,0.02219,21.54,0.2992,0.004877,...,86730502,216.0,NaN,M,NaN,106.2,0.3055,0.01535,0.01952,0.4332
19705,350,2225.0,NaN,0.01162,0.07561,421.0,0.005949,17.07,0.03046,NaN,...,899187,0.1671,0.2731,B,NaN,73.7,0.06476,0.02216,0.006991,0.3534
19706,442,2235.0,15.27,0.009937,NaN,585.9,0.007741,15.79,0.03517,NaN,...,90944601,0.1405,0.1859,B,0.002564,NaN,0.1071,NaN,0.01156,0.3563
19708,501,2974.0,16.01,0.06759,0.1162,595.9,0.03476,24.49,0.3381,0.00968,...,91504,0.2275,0.3651,M,0.006995,92.33,0.3966,0.02434,0.03856,0.4751


In [31]:

import parquet
import fastparquet
df.to_parquet("../data/interim/BreastCancer.parquet", index = False)

In [ ]:
pd.read_parquet("../data/interim/BreastCancer.parquet", engine='pyarrow')